# 对SO数据集分句

In [1]:
#! /usr/bin/env python
import time, datetime, csv, os, codecs, collections, re, string, random, sys, scipy, matplotlib, nltk, nltk.stem
import tensorflow as tf
import numpy as np
import pandas as pd
import nltk.tokenize as tk
import matplotlib.pyplot as plt
from tensorflow.contrib import learn
from sklearn import metrics, feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from gensim import corpora, models, similarities

In [2]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>


False

In [36]:
#remove html tags and links
'''
def remove_tag(content):
    ##remove links
    for ele in re.findall('<a href=.*?</a>', content, re.S):
        content = content.replace(ele, '')
    ##remove codes
    for ele in re.findall('<pre>.*?</pre>', content, re.S):
        content = content.replace(ele, '')
    for ele in re.findall('<code>.*?</code>', content, re.S):
        content = content.replace(ele, '')
    ##remove tags
    for ele in re.findall('</.*?>', content, re.S):
        content = content.replace(ele, '')
    for ele in re.findall('<.*?>', content, re.S):
        content = content.replace(ele, '')
    ##remove '\n' and '\r'
    for ele in re.findall('\n', content, re.S):
        content = content.replace(ele, '')
    for ele in re.findall('\r', content, re.S):
        content = content.replace(ele, '')
    return content
'''

"\ndef remove_tag(content):\n    ##remove links\n    for ele in re.findall('<a href=.*?</a>', content, re.S):\n        content = content.replace(ele, '')\n    ##remove codes\n    for ele in re.findall('<pre>.*?</pre>', content, re.S):\n        content = content.replace(ele, '')\n    for ele in re.findall('<code>.*?</code>', content, re.S):\n        content = content.replace(ele, '')\n    ##remove tags\n    for ele in re.findall('</.*?>', content, re.S):\n        content = content.replace(ele, '')\n    for ele in re.findall('<.*?>', content, re.S):\n        content = content.replace(ele, '')\n    ##remove '\n' and '\r'\n    for ele in re.findall('\n', content, re.S):\n        content = content.replace(ele, '')\n    for ele in re.findall('\r', content, re.S):\n        content = content.replace(ele, '')\n    return content\n"

2019-08-18  
重写remove_tag函数，修复两个句子连在一起的BUG

In [3]:
#remove html tags and links
def remove_tag(content):
    ##remove links
    for ele in re.findall('<a href=.*?</a>', content, re.S):
        content = content.replace(ele, '')
    ##remove codes
    for ele in re.findall('<pre>.*?</pre>', content, re.S):
        content = content.replace(ele, '')
    for ele in re.findall('<code>.*?</code>', content, re.S):
        content = content.replace(ele, '')
    ##remove tags
    for ele in re.findall('</.*?>', content, re.S):
        content = content.replace(ele, ' ')
    for ele in re.findall('<.*?>', content, re.S):
        content = content.replace(ele, ' ')
    ##remove '\n' and '\r'
    for ele in re.findall('\n', content, re.S):
        content = content.replace(ele, ' ')
    for ele in re.findall('\r', content, re.S):
        content = content.replace(ele, ' ')
    return content

# 批量处理与生成

In [5]:
load_path = 'relative_api_data_v1/'
save_title_path = 'API_SO_Titles/'
save_sentence_path = 'API_SO_Sentences/'
file_list = os.listdir(load_path)
len(file_list)

105

In [6]:
file_list

['bigcommerce_R.csv',
 'imgur_R.csv',
 'heroku_R.csv',
 'google-maps-android_R.csv',
 'google-adwords_R.csv',
 'mapbox_R.csv',
 'opensocial_R.csv',
 'sms_R.csv',
 'linkedin_R.csv',
 'wurfl_R.csv',
 'google-adsense_R.csv',
 'digitalocean_R.csv',
 'pubnub_R.csv',
 'payments-gateway_R.csv',
 'google-distance-matrix_R.csv',
 'allegro_R.csv',
 'onelogin_R.csv',
 'amazon-sns_R.csv',
 'stripe_R.csv',
 'musicbrainz_R.csv',
 'plunker_R.csv',
 'vimeo_R.csv',
 'tumblr_R.csv',
 'woocommerce_R.csv',
 'admob_R.csv',
 'itunes-and-itunes-connect_R.csv',
 'deezer_R.csv',
 'mailjet_R.csv',
 'recurly_R.csv',
 'reddit_R.csv',
 'google-geocoding_R.csv',
 'zipcode_R.csv',
 'twitter_R.csv',
 'sendgrid_R.csv',
 'google-checkout_R.csv',
 'google-earth_R.csv',
 'google-picasa_R.csv',
 'openstack_R.csv',
 'breadcrumbs_R.csv',
 '2checkout_R.csv',
 'google-maps_R.csv',
 'gravatar_R.csv',
 'google-cloud-sql_R.csv',
 'vk_R.csv',
 'facebook_R.csv',
 'google-static-maps_R.csv',
 'google-app-engine_R.csv',
 'rhapsody_R

In [8]:
for file in file_list:
    apiname = file[:-6]
    df = pd.DataFrame(pd.read_csv(load_path+file, encoding='ISO-8859-1', header=0))
    print(file, df.shape[0])
    SIds = []
    TIds = []
    Sents = []
    Title = []
    for index in range(0, df.shape[0]):
        if index%1000 == 0:
            print('\tIndex:', index)
        Id = df.iloc[index].Id
        body = df.iloc[index].Body
        title = df.iloc[index].Title
        body = remove_tag(body)
        sentences = tk.sent_tokenize(body)
        for ele in sentences:
            if len(ele) >= 3:
                SIds.append(Id)
                Sents.append(ele)
            else:
                print(ele)
        TIds.append(Id)
        Title.append(title)
    print(len(SIds), len(Sents), len(TIds), len(Title))
    towritet = {'Id':TIds, 'Title':Title}
    towrites = {'Id':SIds, 'Sentence':Sents}
    dataframe = pd.DataFrame(towritet)
    dataframe.to_csv(save_title_path+apiname+'_title.csv',index=False)
    dataframe = pd.DataFrame(towrites)
    dataframe.to_csv(save_sentence_path+apiname+'_sents.csv',index=False)

bigcommerce_R.csv 1276
	Index: 0
.)
};
!
?
2.
3.
4.
?
()
??
?
!!
:(
[
!
!
!
**
.
!
.
.
	Index: 1000
?
.
**
.
?
6292 6292 1276 1276
imgur_R.csv 548
	Index: 0
.
!
:)
2781 2781 548 548
heroku_R.csv 34658
	Index: 0
:D
!
?
?
2.
).
?
?
!
!
!
!
?
?
!
!
?
?
	Index: 1000
:)
:/
Tx
-S
!
!
.
:)
!
.
!
!
:
?
?
:)
?
:)
).
:
?
?
	Index: 2000
1.
2.
2.
DS
!
:D
:
?
)?
?
!
!
!
!
.
".
!
..
!
!
!
:
!
!
?
!
2.
?
	Index: 3000
:)
:)
.
:)
!`
.
.
.
!
!
!
.
?
!
!
!
.
!
:)
 !
).
!
!
!
!
!
!
!
!
C.
	Index: 4000
1.
6.
!
?
.
.
!
?
?
?
:)
!
!
!
!
!
!
?
!
?
	Index: 5000
:D
!
:
!
?
?
).
!
1.
:)
!
):
:)
!
!
2.
).
	Index: 6000
?
!
3.
??
.
?
!
!
:D
!
!
.
.
.
.
!
=)
?
2.
?
!
!
!
:(
	Index: 7000
).
?
!
!
!
!
.
!
).
.
?
()
!
?
!
!
!
!
!
:
!
.
.
`
	Index: 8000
.
:)
?
!
!
!
!
).
?
!
:)
2.
!
?
!
).
!
!
!
?
?
!
:/
?
!
!
!
!
!
	Index: 9000
!
!
!
:(
!
.
.
V
!
!
!
:)
:)
.
?
;)
!
!
!
!
!
.
.
2.
3.
:)
	Index: 10000
!
2.
3.
!
?
B.
1.
1.
2.
).
?
?
1.
2.
:)
".
!
!
!
!
!
!
.
:/
:)
!
?
?
?
	Index: 11000
:D
!
:)
2.
?
!
!
!
!
!
!
).
,
!
.
!


2.
1.
2.
JD
.
2.
:
.
:)
!
	Index: 9000
:)
!
?
!
.
2.
?
2.
?
!
?
	Index: 10000
2.
?
!
?
!
:(
:)
:)
?
?
2.
2.
!
:)
!
:)
Ed
	Index: 11000
!
?
?
).
:O
:)
.
*
!
!
!
!
!
2.
?
.
!
:)
	Index: 12000
.
2.
.
.
?
1.
!
?
?
!
!
!
!
2.
!
:)
!
	Index: 13000
:)
 .
?
?
!
!
!
.
.
!
:)
?
;)
.
.
!
!.
	Index: 14000
?
!
!
!
!
;)
?
!
2.
?
:)
2.
4.
5.
6.
!
:)
1.
2.
3.
?
eg
2.
	Index: 15000
?
?
`
:)
.
!
!
J.
?
!
[
!
!
.
!
	Index: 16000
x
?
!
?
?
!
1.
?
?
?
My
?
J
.
.
:)
?
Sy
2.
	Index: 17000
.
?
?
!
:)
()
?
:)
JC
!
:)
).
?
!
:
?
	Index: 18000
!
:)
:)
:)
:)
!
'?
?
?
?
:)
?
?
?
?
?
2.
!
.
1.
:)
.
	Index: 19000
:)
?
!
=)
!
!
?
?
?
1.
!
?
?>
1.
98685 98685 19668 19668
admob_R.csv 7714
	Index: 0
!
3.
!
?
.
?
?
?
?
1.
2.
3.
4.
!
?
!
.
.
!
(:
!
?
!
 !
?
!
!
?
?
!
.
.
.
.
?
:)
!
?
2.
!
	Index: 1000
/>
!
!
!
!
PC
2.
!
2.
!
!
?
BR
!
.
2.
?
?
!
}
?
?
.
.
?
	Index: 2000
!
*
:)
!
?
?
!
.
:)
:)
"
:)
?
:(
!
?
?
2.
?
?
!
.
.
?
or
!
?
!
	Index: 3000
.
?
!
?
!
!
2.
:D
!
:)
;)
:)
).
;c
?
?
?
:)
4.
?
2.
).
?
?
.
.
:(
).
?
?
I
	Ind

!
!
!
?
:D
.
!
!
.
).
?
.
2.
?
?
Ta
!
!
!
:)
*
	Index: 31000
?
).
?
.
!
?
!
!
.
;)
.
.
.
?
.
WH
:)
?
:)
:D
?
!
.
.
?
b.
!
	Index: 32000
:)
1.
.
.
!
:)
?
!
?
?
?
?
!
?
.
?
?
?
?
?
.
?
Or
?
!
!
2.
.
?
?
2.
4.
!
	Index: 33000
:)
:)
:)
!
?
!
!
!
).
:s
?
!
?
?
?
2.
3.
3.
!
!
2.
!
.
.
.
.
?
?
L.
!
!
1.
	Index: 34000
!
?
?
?
?
:)
!
!
:)
:)
2.
3.
!
!
1.
2.
3.
!
!
!
2.
?
.
.
.
!
	Index: 35000
!
!
m.
!
!
?
!
;)
!
!
?
!.
:)
:
1.
	Index: 36000
.
.
:)
 .
.
:(
:)
!
?
.
.
2.
!
?
?
!
?
:)
!
!
!
?
:)
?
?
 !
:/
!
!
!
?
	Index: 37000
".
!
!
?
!
!
?
).
!
:)
!
.
?
?
:
!
!
2.
?
.
2.
?
:(
?
?
2.
?
!
	Index: 38000
?
2.
!
:)
!
:(
:D
?
?
.
:)
1.
?
!
).
?
?
?
?
.
?
!
!
!
:)
!
?
?
!
!
?
?
!
	Index: 39000
.
?
!
.
.
};
?
:)
L.
?
1.
2.
?
?
`
2.
.
.
:
	Index: 40000
?
!
!
?
.
?
:)
:
:(
!
!
is
.
.
.
.
!
?
!
:(
:(
.
?
.
	Index: 41000
:D
?
!
).
2.
!
!
?
b.
!
:(
:
!
?
).
??
?
!
	Index: 42000
!
2.
?
1.
2.
3.
4.
!
1.
!
!
.
.
?
!
.
Ty
}
C
!
.
:)
?
?
?
?
?
?
	Index: 43000
?
",
2.
!
?
1.
2.
3.
:(
:)
2.
3.
.
**
?
K
-
!
:D
2.
3.

?
?
:)
?
.
:)
!
2.
3.
.
.
.
!
?
	Index: 59000
!
".
!
!
?
!
?
:
.
I.
!
?
??
?
.
?
.
!
?
:)
!
!
!
?
!
?
C.
2.
3.
.
	Index: 60000
?
Tx
?
).
?
?
:)
JS
?>
!
.
?
?
?
?
2.
:
:O
?
!
:)
!
!
:)
?
?
:D
2.
3.
	Index: 61000
!
!
=D
!
!
:)
!
.
`
AB
3.
!
?
:(
!
?
?
6.
!
?
	Index: 62000
!
?
2.
3.
4.
?
?
?
?
:)
!!
?
?
!
?
?
?
.
.
:)
.
-M
PK
?
!
b.
!
!
!!
?
?
?
1.
--
	Index: 63000
.
!
?
?
?
.
:)
2.
3.
?
?
:)
!
:)
?>
.
.
?
?
a.
!
!
2.
?
Ry
!
S.
2.
!
!
?
1.
2.
3.
2.
3.
!
	Index: 64000
2.
?
!
=)
2.
},
?
:)
!
?
.
?
?
.
.
?
.
:(
!
?
?
	Index: 65000
);
2.
?
!
KR
!
?
!
2.
3.
?
?
.
!
?
?
?
):
?
:)
?
?
?
:(
?
?
!
!
BR
.
!
?
:)
!
	Index: 66000
.
?
1.
2.
.
.
.
.
.
.
.
!
..
?
!
?
:)
!
:/
!
:
!
!
).
.
?
).
).
3.
:(
?
	Index: 67000
!
!
:)
!
!
?
!
?
^^
?
?
?
!
?
!
?
!
.
:
?
!
"?
.
.
.
JS
?
!
.
.
.
.
?
!
	Index: 68000
2.
?
?
2.
D.
:
!
:)
?
!
!
?
!
:)
.
1.
2.
!
!
?
?
.
!
	Index: 69000
!
:)
?
.
?
!
.
?
.
2.
?
:)
:
.
".
?
!
2.
.
.
!
?
.
:
?
!
.
!
:)
/a
	Index: 70000
:)
!
:(
?
:)
?
?
S.
.
?
:)
.
:)
!
?
k.
:)
..
!
	Index: 71

.
	Index: 8000
!
:)
!
?
:
?
?
**
:)
.
?
?
?
?
?
?
.
 !
?
!
?
!
!
	Index: 9000
?
?
:)
?
4.
6.
!
JS
?
?
1.
2.
?
.
.
!
2.
3.
?
:(
!
">
:D
.
JS
;)
)
?
	Index: 10000
.
.
?
?
**
?
!
?
?
).
(:
?
:D
:
!
!
.
.
.
.
	Index: 11000
?
?
?
!
2.
2.
?
:(
.
!
2.
?
?
!
:)
?
?
!
:(
:)
?
.
.
!
?
	Index: 12000
?
!
.
!
!
!
?
?
;)
!
).
?
?
).
?
2.
?
?
	Index: 13000
).
.
?
!
:)
!
?
.
!
?
	Index: 14000
!
?
!
2.
1?
!
.
!
!
!
:)
!
()
.
	Index: 15000
!
!
:1
.
JS
1.
2.
?
?
:)
?
:/
.
.
.
?
.
:)
.
.
.
.
.
!
?
.
?
??
?
?
.
.
.
!
?
79158 79158 15890 15890
eventbrite_R.csv 272
	Index: 0
:)
!
1.
2.
!
1258 1258 272 272
openx_R.csv 201
	Index: 0
?
!
.
1059 1059 201 201
twilio_R.csv 3200
	Index: 0
:)
!
!
:
?
.
!
)?
2.
?
	Index: 1000
!
2.
2.
3.
?
3.
1.
4.
5.
?>
1.
2.
).
-
!
?
Ta
)?
?
!
).
).
	Index: 2000
!
?
1.
2.
:)
.
.
)?
!
?
.
!
()
!
!
?
!
!
!
	Index: 3000
1.
?
!
15652 15652 3200 3200
openstreetmap_R.csv 3246
	Index: 0
!
:D
?
?
:)
2.
2.
?
Kr
.
.
.
.
!
)?
1.
:P
:)
!
!
!
	Index: 1000
.
?
1.
!
?
?
?
.
.
!
:)
2.
:)
!
.
?
?
?


?
?
`
	Index: 14000
?
.
?
?
2.
3.
!
?
?
?
D.
!
.
!
).
!
?
!
:
	Index: 15000
!
!
?
2.
3.
:)
?
.
!
.
?
2.
3.
?
:(
?
?
.
.
.
.
.
.
.
.
.
.
.
?
!
.
	Index: 16000
)?
).
?
?
?
!
1.
2.
3.
?
?
:)
?
!
.
1.
2.
!
?
.
1.
).
	Index: 17000
.
?
.
!
2.
?
?
?
!
?
.
?
?
?
?
!
!
.
.
.
.
.
.
.
!
?
	Index: 18000
.
?
T
!
?
1.
2.
.
?
?
?
.
?
?
).
1.
2.
!
:(
	Index: 19000
1.
2.
".
!
).
!
.
!
2.
?
!
!
?
?
!
3.
	Index: 20000
?
2.
2.
:)
v
!
ge
:D
?
or
?
T.
!
2.
3.
1.
.
?
?
".
:)
:)
).
!
	Index: 21000
!
?
!
:
.
?
?
?
).
A
!
!
?
?
*/
!
	Index: 22000
?
2.
.
SP
.
1.
!
):
.
!
?
!
:)
)?
?>
'.
?
3.
	Index: 23000
!
!
:)
).
?
?
:(
2.
!
1.
!
?
:
).
.
	Index: 24000
??
?
.
.
2.
b.
".
2.
?
!
.)
!
?
!
.
?
	Index: 25000
!
?
2.
?
!
!
!
.
.
.
.
?
!
!
:(
?
!
!
	Index: 26000
?
.
?
!
!
p
?
).
?
?
:)
..
?
?
!
!
2.
3.
4.
	Index: 27000
.
!
.
}`
.
?
?
!
?
2.
3.
.
?
:)
	Index: 28000
!
?
!
.
?
?
?
!
?
).
!
?
	Index: 29000
!
.
.
?
!
?
3.
()
!
2.
()
2.
).
?
!
:)
171304 171304 29857 29857
flickr_R.csv 1638
	Index: 0
?
?
!
:
!
!
).
.
.
1.
2.